In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb 
import xgboost as xgb 
import catboost as cbt 
import numpy as np 
import joblib 
import os 
from sklearn.metrics import mean_absolute_error
import tensorflow as tf
from sklearn.model_selection import GroupKFold
from itertools import combinations
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import polars as pl
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import Pool, CatBoostRegressor
from tqdm import tqdm
print('Tensorflow version',tf.__version__)

/usr/local/lib/python3.8/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-10-29 21:50:52.221494: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-29 21:50:52.766471: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Tensorflow version 2.13.1


In [2]:
NUMS = [ 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap','bid_ask_imb_minus', 'imbalance_match_imb_minus']
prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']

gaps = [i for i in range(1, 56)]

In [4]:
columns_convert = [pl.col(i).cast(pl.Float32) for i in NUMS if i not in ('bid_ask_imb_minus', 'imbalance_match_imb_minus')] 
columns_convert += [(((pl.col('bid_size') - pl.col('ask_size')) ).alias(f'bid_ask_imb_minus')),
                    (((pl.col('imbalance_size') - pl.col('matched_size')) ).alias(f'imbalance_match_imb_minus')),
                   
                   ]
columns = [] 

for i,a in enumerate(prices):
    for j,b in enumerate(prices):
        if i>j:
            columns.append( (((pl.col(a) - pl.col(b))/(pl.col(a)+pl.col(b)) ).alias(f'{a}_{b}_imb')) )
            columns.append( (((pl.col(a) - pl.col(b)) ).alias(f'{a}_{b}_imb_minus')) )
            
columns_part2 = [
    
    
    #**[pl.col("imbalance_size").shift(i).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias(f"imbalance_size_{i}") for i in range(1, 6)],
    
    
#     (( pl.col("imbalance_size").shift(1)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("imbalance_size_0")),
#     (( pl.col("imbalance_size").shift(2)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("imbalance_size_1")),
#     (( pl.col("imbalance_size").shift(3)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("imbalance_size_2")),
#     (( pl.col("imbalance_size").shift(4)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("imbalance_size_3")),
#     (( pl.col("imbalance_size").shift(5)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("imbalance_size_4")),
#     (( pl.col("imbalance_size").shift(6)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("imbalance_size_5")),
    
    
    ((pl.col("imbalance_size") - pl.col("imbalance_size").shift(1)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("imbalance_size_diff_0")),
    ((pl.col("imbalance_size") - pl.col("imbalance_size").shift(2)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("imbalance_size_diff_1")),
    ((pl.col("imbalance_size") - pl.col("imbalance_size").shift(3)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("imbalance_size_diff_2")),
    ((pl.col("imbalance_size") - pl.col("imbalance_size").shift(4)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("imbalance_size_diff_3")),
    ((pl.col("imbalance_size") - pl.col("imbalance_size").shift(5)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("imbalance_size_diff_4")),
    ((pl.col("imbalance_size") - pl.col("imbalance_size").shift(6)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("imbalance_size_diff_5")),
    
    
    ((pl.col("matched_size") - pl.col("matched_size").shift(1)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("matched_size_diff_0")),
    ((pl.col("matched_size") - pl.col("matched_size").shift(2)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("matched_size_diff_1")),
    ((pl.col("matched_size") - pl.col("matched_size").shift(3)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("matched_size_diff_2")),
    ((pl.col("matched_size") - pl.col("matched_size").shift(4)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("matched_size_diff_3")),
    ((pl.col("matched_size") - pl.col("matched_size").shift(5)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("matched_size_diff_4")),
    ((pl.col("matched_size") - pl.col("matched_size").shift(6)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("matched_size_diff_5")),
    
    ((pl.col("bid_size") - pl.col("bid_size").shift(1)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("bid_size_diff_0")),
    ((pl.col("bid_size") - pl.col("bid_size").shift(2)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("bid_size_diff_1")),
    ((pl.col("bid_size") - pl.col("bid_size").shift(3)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("bid_size_diff_2")),
    ((pl.col("bid_size") - pl.col("bid_size").shift(4)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("bid_size_diff_3")),
    ((pl.col("bid_size") - pl.col("bid_size").shift(5)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("bid_size_diff_4")),
    ((pl.col("bid_size") - pl.col("bid_size").shift(6)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("bid_size_diff_5")),

    ((pl.col("ask_size") - pl.col("ask_size").shift(1)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("ask_size_diff_0")),
    ((pl.col("ask_size") - pl.col("ask_size").shift(2)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("ask_size_diff_1")),
    ((pl.col("ask_size") - pl.col("ask_size").shift(3)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("ask_size_diff_2")),
    ((pl.col("ask_size") - pl.col("ask_size").shift(4)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("ask_size_diff_3")),
    ((pl.col("ask_size") - pl.col("ask_size").shift(5)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("ask_size_diff_4")),
    ((pl.col("ask_size") - pl.col("ask_size").shift(6)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("ask_size_diff_5")),
    
    ((pl.col("bid_ask_imb_minus") - pl.col("bid_ask_imb_minus").shift(1)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("bid_ask_imb_minus_diff_0")),
    ((pl.col("bid_ask_imb_minus") - pl.col("bid_ask_imb_minus").shift(2)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("bid_ask_imb_minus_diff_1")),
    ((pl.col("bid_ask_imb_minus") - pl.col("bid_ask_imb_minus").shift(3)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("bid_ask_imb_minus_diff_2")),
    ((pl.col("bid_ask_imb_minus") - pl.col("bid_ask_imb_minus").shift(4)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("bid_ask_imb_minus_diff_3")),
    ((pl.col("bid_ask_imb_minus") - pl.col("bid_ask_imb_minus").shift(5)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("bid_ask_imb_minus_diff_4")),
    ((pl.col("bid_ask_imb_minus") - pl.col("bid_ask_imb_minus").shift(6)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("bid_ask_imb_minus_diff_5")),
    
    ((pl.col("imbalance_match_imb_minus") - pl.col("imbalance_match_imb_minus").shift(1)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("imbalance_match_imb_minus_diff_0")),
    ((pl.col("imbalance_match_imb_minus") - pl.col("imbalance_match_imb_minus").shift(2)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("imbalance_match_imb_minus_diff_1")),
    ((pl.col("imbalance_match_imb_minus") - pl.col("imbalance_match_imb_minus").shift(3)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("imbalance_match_imb_minus_diff_2")),
    ((pl.col("imbalance_match_imb_minus") - pl.col("imbalance_match_imb_minus").shift(4)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("imbalance_match_imb_minus_diff_3")),
    ((pl.col("imbalance_match_imb_minus") - pl.col("imbalance_match_imb_minus").shift(5)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("imbalance_match_imb_minus_diff_4")),
    ((pl.col("imbalance_match_imb_minus") - pl.col("imbalance_match_imb_minus").shift(6)).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias("imbalance_match_imb_minus_diff_5")),
    
    ( (  (pl.col("matched_size") - pl.col("imbalance_size")) / (pl.col("matched_size") + pl.col("imbalance_size"))  ).alias("matched_imblance_diff_0") ),
    ( (  (pl.col("matched_size") - pl.col("imbalance_size"))   ).alias("matched_imblance_diff_1") ),
    # bid_size-ask_size
    ( (pl.col("bid_size") - pl.col("ask_size")).alias("bid_ask_diff_0") ),
    ( (  (pl.col("bid_size") - pl.col("ask_size")) / (pl.col("bid_size") + pl.col("ask_size"))  ).alias("bid_ask_diff_1") ),
]


mins = 60
day = 1
year = (365.2425)*day

columns_part3 = [

    # ( np.sin((pl.col("date_id") + 1) * (2 * np.pi / day)).alias("day_sin") ),
    # ( np.cos((pl.col("date_id") + 1) * (2 * np.pi / day)).alias("day_cos") ),
#     ( np.sin((pl.col("date_id") + 1) * (2 * np.pi / year)).alias("year_sin") ),
#     ( np.cos((pl.col("date_id") + 1) * (2 * np.pi / year)).alias("year_cos") ),
    
#     ( np.sin((pl.col("seconds_in_bucket")) * (2 * np.pi / mins)).alias("min_sin") ),
#     ( np.cos((pl.col("seconds_in_bucket") ) * (2 * np.pi / mins)).alias("min_cos") ),
    
    ( np.sin((pl.col("seconds_in_bucket")) * (2 * np.pi / 1)).alias("second_sin") ),
    ( np.cos((pl.col("seconds_in_bucket") ) * (2 * np.pi / 1)).alias("second_cos") ),
]

columns_part4 = []
for num in NUMS:
    for gap in gaps:
        columns_part4.append(
            
        pl.col(num).shift(gap).fill_null(0).clip(-1e9, 1e9).over(["stock_id", "date_id"]).alias(f"{num}_gap_{gap}")
        
        )


columns += columns_part2 + columns_part4 + columns_part3

In [5]:
df = (pl.read_csv("train.csv")
      .with_columns(columns_convert)
      .with_columns(columns)
      .drop(['row_id', 'time_id'])
     )

In [6]:
df.filter((pl.col('stock_id')==0))

stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,bid_ask_imb_minus,imbalance_match_imb_minus,far_price_reference_price_imb,far_price_reference_price_imb_minus,near_price_reference_price_imb,near_price_reference_price_imb_minus,near_price_far_price_imb,near_price_far_price_imb_minus,ask_price_reference_price_imb,ask_price_reference_price_imb_minus,ask_price_far_price_imb,ask_price_far_price_imb_minus,ask_price_near_price_imb,ask_price_near_price_imb_minus,bid_price_reference_price_imb,bid_price_reference_price_imb_minus,bid_price_far_price_imb,bid_price_far_price_imb_minus,bid_price_near_price_imb,bid_price_near_price_imb_minus,bid_price_ask_price_imb,bid_price_ask_price_imb_minus,...,imbalance_match_imb_minus_gap_21,imbalance_match_imb_minus_gap_22,imbalance_match_imb_minus_gap_23,imbalance_match_imb_minus_gap_24,imbalance_match_imb_minus_gap_25,imbalance_match_imb_minus_gap_26,imbalance_match_imb_minus_gap_27,imbalance_match_imb_minus_gap_28,imbalance_match_imb_minus_gap_29,imbalance_match_imb_minus_gap_30,imbalance_match_imb_minus_gap_31,imbalance_match_imb_minus_gap_32,imbalance_match_imb_minus_gap_33,imbalance_match_imb_minus_gap_34,imbalance_match_imb_minus_gap_35,imbalance_match_imb_minus_gap_36,imbalance_match_imb_minus_gap_37,imbalance_match_imb_minus_gap_38,imbalance_match_imb_minus_gap_39,imbalance_match_imb_minus_gap_40,imbalance_match_imb_minus_gap_41,imbalance_match_imb_minus_gap_42,imbalance_match_imb_minus_gap_43,imbalance_match_imb_minus_gap_44,imbalance_match_imb_minus_gap_45,imbalance_match_imb_minus_gap_46,imbalance_match_imb_minus_gap_47,imbalance_match_imb_minus_gap_48,imbalance_match_imb_minus_gap_49,imbalance_match_imb_minus_gap_50,imbalance_match_imb_minus_gap_51,imbalance_match_imb_minus_gap_52,imbalance_match_imb_minus_gap_53,imbalance_match_imb_minus_gap_54,imbalance_match_imb_minus_gap_55,second_sin,second_cos
i64,i64,i64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,...,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,0,0,3.1806e6,1.0,0.999812,1.3380277e7,null,null,0.999812,60651.5,1.000026,8493.030273,1.0,-3.029704,52158.47,-1.0200e7,null,null,null,null,null,null,0.000107,0.000214,null,null,null,null,0.0,0.0,null,null,null,null,-0.000107,-0.000214,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
0,0,10,1.2998e6,1.0,1.000026,1.5261107e7,null,null,0.999812,13996.5,1.000026,23519.160156,0.999892,0.389814,-9522.66,-1.3961e7,null,null,null,null,null,null,0.0,0.0,null,null,null,null,-0.000107,-0.000214,null,null,null,null,-0.000107,-0.000214,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.4493e-15,1.0
0,0,20,1.2998e6,1.0,0.999919,1.5261107e7,null,null,0.999812,4665.5,0.999919,12131.599609,0.999842,4.220009,-7466.1,-1.3961e7,null,null,null,null,null,null,0.0,0.0,null,null,null,null,-0.000054,-0.000107,null,null,null,null,-0.000054,-0.000107,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.8986e-15,1.0
0,0,30,1.2998e6,1.0,1.000133,1.5261107e7,null,null,1.000026,55998.0,1.000133,46203.300781,1.000085,5.4502487,9794.7,-1.3961e7,null,null,null,null,null,null,0.0,0.0,null,null,null,null,-0.000054,-0.000107,null,null,null,null,-0.000054,-0.000107,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.1559e-14,1.0
0,0,40,1.2182e6,1.0,1.000455,1.5342675e7,null,null,1.000241,14655.950195,1.000455,26610.449219,1.000317,3

In [7]:
df.filter((pl.col('stock_id')==0)&(pl.col('date_id')==0)).select(['imbalance_size', 'imbalance_size_diff_0'])

imbalance_size,imbalance_size_diff_0
f32,f32
3.1806e6,0.0
1.2998e6,-1.88083e6
1.2998e6,0.0
1.2998e6,0.0
1.2182e6,-81568.375
1.2182e6,0.0
1.2182e6,0.0
1.2645e6,46290.5
1.1898e6,-74662.0


In [8]:
NUMS += [i for i in df.columns if i not in ['stock_id', 'date_id', 'seconds_in_bucket'] and i not in NUMS]
NUMS

['imbalance_size',
 'imbalance_buy_sell_flag',
 'reference_price',
 'matched_size',
 'far_price',
 'near_price',
 'bid_price',
 'bid_size',
 'ask_price',
 'ask_size',
 'wap',
 'bid_ask_imb_minus',
 'imbalance_match_imb_minus',
 'target',
 'far_price_reference_price_imb',
 'far_price_reference_price_imb_minus',
 'near_price_reference_price_imb',
 'near_price_reference_price_imb_minus',
 'near_price_far_price_imb',
 'near_price_far_price_imb_minus',
 'ask_price_reference_price_imb',
 'ask_price_reference_price_imb_minus',
 'ask_price_far_price_imb',
 'ask_price_far_price_imb_minus',
 'ask_price_near_price_imb',
 'ask_price_near_price_imb_minus',
 'bid_price_reference_price_imb',
 'bid_price_reference_price_imb_minus',
 'bid_price_far_price_imb',
 'bid_price_far_price_imb_minus',
 'bid_price_near_price_imb',
 'bid_price_near_price_imb_minus',
 'bid_price_ask_price_imb',
 'bid_price_ask_price_imb_minus',
 'wap_reference_price_imb',
 'wap_reference_price_imb_minus',
 'wap_far_price_imb',
 '

## featuring engineering

In [9]:
eps = 1e-3
def feature_engineer_for_index(x, date, time):
    #x = x.filter(((pl.col('seconds_in_bucket') <= time) & (pl.col('date_id') == date)) | (pl.col('date_id') < date) )
    x = x.filter(((pl.col('seconds_in_bucket') <= time) & (pl.col('date_id') == date)) )
    feature_suffix = 'part_1'
    aggs = [
        *[pl.col(c).quantile(0.1, "nearest").alias(f"{c}_quantile1_{feature_suffix}") for c in NUMS],
        *[pl.col(c).quantile(0.2, "nearest").alias(f"{c}_quantile2_{feature_suffix}") for c in NUMS],
        *[pl.col(c).quantile(0.3, "nearest").alias(f"{c}_quantile3_{feature_suffix}") for c in NUMS],
        *[pl.col(c).quantile(0.4, "nearest").alias(f"{c}_quantile4_{feature_suffix}") for c in NUMS],
        *[pl.col(c).quantile(0.5, "nearest").alias(f"{c}_quantile5_{feature_suffix}") for c in NUMS],
        *[pl.col(c).quantile(0.6, "nearest").alias(f"{c}_quantile6_{feature_suffix}") for c in NUMS],
        *[pl.col(c).quantile(0.7, "nearest").alias(f"{c}_quantile7_{feature_suffix}") for c in NUMS],
        *[pl.col(c).quantile(0.8, "nearest").alias(f"{c}_quantile8_{feature_suffix}") for c in NUMS],
        *[pl.col(c).quantile(0.9, "nearest").alias(f"{c}_quantile9_{feature_suffix}") for c in NUMS],
        *[pl.col(c).mean().alias(f"{c}_mean_{feature_suffix}") for c in NUMS],
        *[pl.col(c).std().alias(f"{c}_std_{feature_suffix}") for c in NUMS],
        *[pl.col(c).min().alias(f"{c}_min_{feature_suffix}") for c in NUMS],
        *[pl.col(c).max().alias(f"{c}_max_{feature_suffix}") for c in NUMS],
        *[pl.col(c).median().alias(f"{c}_median_{feature_suffix}") for c in NUMS],
        *[pl.col(c).sum().alias(f"{c}_sum_{feature_suffix}") for c in NUMS],
        *[pl.col(c).skew().alias(f"{c}_skew_{feature_suffix}") for c in NUMS],
        *[pl.col(c).kurtosis().alias(f"{c}_kurtosis_{feature_suffix}") for c in NUMS],
        # adding rank features
        *[pl.col(c).last().alias(f"{c}_rank_{feature_suffix}") for c in NUMS],
        ]

    features_part_1= x.groupby(["stock_id"], maintain_order=True).agg(aggs)
    
    x = x.filter((pl.col('seconds_in_bucket') == time) & (pl.col('date_id') == date))
    df = x.join(features_part_1, on='stock_id')
    # df = df.join(features_part_2, on=['stock_id', 'seconds_in_bucket'])
    # df = df.join(features_part_3, on=['seconds_in_bucket'])
    #df = df.filter((pl.col('seconds_in_bucket') == time) & (pl.col('date_id') == date))
    #df = df.to_pandas()
    
    return df

In [ ]:
train = []
for date in tqdm(range(481)):
    for t in range(0, 550, 10):
        train.append(feature_engineer_for_index(df, date, t))
train = pl.concat(train)
train = train.to_pandas()
train.dropna(subset=['target'], inplace=True)

 62%|██████▏   | 296/481 [1:06:07<43:25, 14.09s/it]

In [ ]:
# some cleaning...
null = train.isnull().sum().sort_values(ascending=False) / len(train)


drop = list(null[null>0.9].index)


for col in train.columns:
    if train[col].nunique()==1:
        drop.append(col)
print("*********df DONE*********")

In [ ]:
FEATURES = [c for c in train.columns if c not in drop + ['target', 'date_id']] 

In [ ]:
len(FEATURES)

## train & val split

In [ ]:
train_df = train[train['date_id']< 450]
val_df = train[train['date_id'] >= 450 ]

In [ ]:
train_df.shape

In [ ]:
train_x, train_y = train_df[FEATURES], train_df[['target']]
valid_x, valid_y = val_df[FEATURES], val_df[['target']]

In [ ]:
xgb_params = {
        'booster': 'gbtree',
        'tree_method': 'gpu_hist',
        'objective': 'reg:squarederror',
        'eval_metric':'mae',
        'learning_rate': 0.02,
        'alpha': 8,
        'max_depth': 4,
        'subsample':0.8,
        'colsample_bytree': 0.5,
        'seed': 42
        }

xgb_params['n_estimators'] = 500


lgbm_params = {
    'objective' : 'regression_l1',
    'num_iterations': 500,
 
}

In [ ]:
model_name = 'lgbm'

feature_importance_df = pd.DataFrame()
if model_name == 'lgbm':
    clf = LGBMRegressor(**lgbm_params) 
else:
    clf = XGBRegressor(**xgb_params)
clf.fit(train_x.astype('float32'), train_y['target'],
        eval_set=[(valid_x.astype('float32'), valid_y['target'])],
        verbose=0)

if model_name == 'lgbm':
    clf.booster_.save_model(f'./lgbm_5fold/lgbm.xgb')
elif model_name == 'xgb':
    clf.save_model(f'./xgb_5fold/XGB.xgb')
else:
    clf.save_model(f'./cat_5fold/cat.xgb')
#best_iteration_xgb[str(i)].append(clf.best_ntree_limit)


fold_importance_df = pd.DataFrame()
fold_importance_df["feature"] = FEATURES
fold_importance_df["importance"] = clf.feature_importances_
fold_importance_df["fold"] = 1
feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)


feature_importance_df = feature_importance_df.groupby(['feature'])['importance'].agg(['mean']).sort_values(by='mean', ascending=False)
display(feature_importance_df.head(30))

## eval

In [ ]:
mean_absolute_error(valid_y['target'], clf.predict(valid_x))

In [29]:
mean_absolute_error(valid_y['target'], clf.predict(valid_x))

5.8080751022665185

In [1]:
!ls

example_test_files  optiver-trading-at-the-close.zip   train_gru.ipynb
features.npy	    public_timeseries_testing_util.py  xgb_5fold
lgbm_5fold	    public_train_baseline.ipynb        xgb_5fold.zip
lgbm.zip	    train_baseline.ipynb
optiver2023	    train.csv


In [2]:
!wc -l ./example_test_files/sample_submission.csv

33001 ./example_test_files/sample_submission.csv


In [ ]:
# baseline 5.90734 
# light gbm 5.8321366685330425
# cat boost 5.836515258358157

## retraining total

In [32]:
np.save('./lgbm_5fold/features', FEATURES)

In [30]:
model_name = 'lgbm'
train_x, train_y = train[FEATURES], train[['target']]
feature_importance_df = pd.DataFrame()
if model_name == 'lgbm':
    clf = LGBMRegressor(**lgbm_params) 
else:
    clf = XGBRegressor(**xgb_params)
clf.fit(train_x.astype('float32'), train_y['target'],
        verbose=0)

if model_name == 'lgbm':
    clf.booster_.save_model(f'./lgbm_5fold/final_lgbm.xgb')
elif model_name == 'xgb':
    clf.save_model(f'./xgb_5fold/final_XGB.xgb')
else:
    clf.save_model(f'./cat_5fold/final_cat.xgb')
#best_iteration_xgb[str(i)].append(clf.best_ntree_limit)



/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [33]:
!zip -r lgbm.zip lgbm_5fold

updating: lgbm_5fold/ (stored 0%)
updating: lgbm_5fold/lgbm.xgb (deflated 58%)
updating: lgbm_5fold/final_lgbm.xgb (deflated 58%)
updating: lgbm_5fold/features.npy (deflated 96%)
